In [ ]:
import numpy as np
from networkx import nx
from collections import defaultdict

import pandas as pd
import datetime
import matplotlib.pyplot as plt

In [ ]:
edgesAfter = pd.read_csv("../data/test.csv")
# edgesAfter = pd.read_csv("../data/votes_to_comments_after.csv")

In [ ]:
edges = edgesAfter.head(50)

In [ ]:
G = nx.from_pandas_edgelist(edges, 
                            source='source', 
                            target='target', 
                            edge_attr = 'weight',
                            create_using=nx.DiGraph())

In [ ]:
fig = plt.figure(figsize=(50,50))
nx.draw_spring(G)
plt.show()

In [ ]:
x = nx.triadic_census(G)
x

In [ ]:
def triads_by_specific_types(G, my_types):
    """Returns a list of triads of pre-defined types in a directed graph.
    Parameters
    ----------
    G : digraph
       A NetworkX DiGraph
    Returns
    -------
    tri_by_type : dict
       Dictionary with triad types as keys and lists of triads as values.
    """
    all_tri = nx.all_triads(G)
    tri_by_type = defaultdict(list)
    for triad in all_tri:
        if (nx.triad_type(triad) in my_types):
            name = nx.triad_type(triad)
            tri_by_type[name].append(triad)
    return tri_by_type

In [ ]:
#test = triads_by_specific_types(G, my_types = {'030T', '120D', '120U', '300', '021D'})
#(test['021D'])[:5]

In [ ]:
def balance_ratio(graph):
    my_types = {'030T', '120D', '120U', '300'}
    triads = triads_by_specific_types(graph, my_types)
    
    sum_balance_030T = 0
    sum_balance_120D = 0
    sum_balance_120U = 0
    sum_balance_300 = 0
    
    for triad_type in my_types:
        for i in range(len(triads[triad_type])):
            
            if (triad_type == '030T'):
                edge_data = ((triads['030T'])[i]).edges(data = True)
                semicycle_balance_030T = 1
                for edge in edge_data:
                    semicycle_balance_030T *= (edge[2])['weight']
                if (semicycle_balance_030T > 0):
                    sum_balance_030T += 1                   
                else:
                    sum_balance_030T += 0
                
                
            elif (triad_type == '120D'):
                edge_data = ((triads['120D'])[i]).edges(data = True)
                triangle_balance_120D = balance_for_2_semicycles(edge_data)
                
                sum_balance_120D += triangle_balance_120D
                
            elif (triad_type == '120U'):
                edge_data = ((triads['120U'])[i]).edges(data = True)
                triangle_balance_120U = balance_for_2_semicycles(edge_data)
                
                sum_balance_120U += triangle_balance_120U
                
            
            #elif (triad_type == '300'):
                #balance_300 =
            
    
    overall_balance_030T = sum_balance_030T / len(triads['030T'])
    overall_balance_120D = sum_balance_120D / len(triads['120D'])
    overall_balance_120U = sum_balance_120U / len(triads['120U'])
    overall_balance_300 = sum_balance_300 / len(triads['300'])
    
    average_overall_balance = (overall_balance_030T + overall_balance_120D + overall_balance_120U + overall_balance_300) / 4
    return average_overall_balance
    
    

In [ ]:
triades = triads_by_specific_types(G, {'030T', '120D', '120U', '300'})
test_120D = ((triades['120D'])[1]).edges(data = True)
test_120D


In [ ]:
def balance_for_2_semicycles(triad_edge_data):
    semicycle_balance = 1
    x, y = find_double_edge(triad_edge_data)
    positive_set = []
    negative_set = []
    
    # calculating balance/imbalance for both semicycles
    for edge in triad_edge_data:
        semicycle_balance *= (edge[2])['weight']
        
    for edge in triad_edge_data:
        if ((x == edge[0]) and (y == edge[1])):
            semicycle_balance_1 = semicycle_balance / ((edge[2])['weight'])
        if ((x == edge[1]) and (y == edge[0])):
            semicycle_balance_2 = semicycle_balance / ((edge[2])['weight'])
            
    # checking for both semicycles if balanced or imbalanced   
    if (semicycle_balance_1 > 0):
        positive_set.append(semicycle_balance_1)
    else:
        negative_set.append(semicycle_balance_1)
            
    if (semicycle_balance_2 > 0):
        positive_set.append(semicycle_balance_2)
    else:
        negative_set.append(semicycle_balance_2)
            
    triangle_balance = len(positive_set)/(len(positive_set) + len(negative_set))   # step 8 in algorithm             
    return triangle_balance
        

    

In [ ]:
def find_double_edge(triad_edge_data):
    for i in triad_edge_data:
        for j in triad_edge_data:
            if (i[0]==j[1] and i[1]==j[0]):
                x = i[0]
                y = i[1]
    return x,y

In [ ]:
x,y = find_double_edge(test_120D)
x, y

In [ ]:
balance_for_2_semicycles(test_120D)

In [ ]:
y = nx.triads_by_type(G)
blubb = (y['120D'])[0].edges(data = True)
blubb

In [ ]:
((y['120D'])[0].get_edge_data(2, 5))['weight']

In [ ]:
balance_ratio(G)